In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
# https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkConf
config = SparkConf()
config.set("spark.driver.memory", "2g")
config.set("spark.executor.memory", "1g")
spark = SparkSession.builder.config(conf=config).master("local").appName("Analyzing Real Estate Sales").getOrCreate()

In [2]:
df = spark.read.format('csv').option("header", "true").load('../Real_Estate_Sales_2001-2017.csv')

# Spark dataframe to pandas

In [3]:
# https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
pandas_df = df.select("*").toPandas()
pandas_df

ID SerialNumber ListYear            DateRecorded        Town  \
0       815906       170177     2017  04/05/1999 12:00:00 AM  New London   
1            2       900035     2009  07/20/2010 12:00:00 AM     Andover   
2            3        14011     2014  01/14/2015 12:00:00 AM     Andover   
3            4        80009     2008  01/21/2009 12:00:00 AM     Andover   
4            5        15006     2015  11/30/2015 12:00:00 AM     Andover   
...        ...          ...      ...                     ...         ...   
861592  610215       162032     2016  09/20/2017 12:00:00 AM    Stamford   
861593   79276       150686     2015  07/29/2016 12:00:00 AM     Bristol   
861594  731402        70105     2007  03/10/2008 12:00:00 AM   Watertown   
861595  734206        10411     2001  09/23/2002 12:00:00 AM   Watertown   
861596  524390        40319     2004  07/18/2005 12:00:00 AM  Plainville   

                  Address AssessedValue SaleAmount         SalesRatio  \
0          293 PEQUOT AVE        132440     252500              0.525   
1         1 DOGWOOD DRIVE         55600      99000  0.561616161616162   
2         1 JUROVATY LANE        153100     190000        0.805789474   
3             1 ROSE LANE        116600     138900  0.839452843772498   
4             1 ROSE LANE        102900      50000              2.058   
...                   ...           ...        ...                ...   
861592     143 ALTON ROAD        226120     480000        0.471083333   
861593    71-79 MEADOW ST        263620     530000        0.497396226   
861594  116 PINE RIDGE DR         62500     385000  0.162337662337662   
861595     56 COLONIAL ST         95800     189900   50.4476040021064   
861596        300 CAMP ST        179050     396500   45.1576292559899   

       PropertyType ResidentialType        NonUseCode  \
0              None      Two Family              None   
1       Vacant Land              NA                NA   
2       Residential   Single Family                NA   
3       Residential   Single Family                NA   
4       Residential   Single Family  14 - Foreclosure   
...             ...             ...               ...   
861592  Residential   Single Family                NA   
861593   Commercial              NA                NA   
861594  Residential   Single Family                 7   
861595  Residential    Three Family                 3   
861596  Residential   Single Family                 0   

                               Remarks  
0                                 None  
1                                   NA  
2                                   NA  
3                                   NA  
4       PROPERTY WAS OWNED BY THE BANK  
...                                ...  
861592                              NA  
861593                              NA  
861594                              NA  
861595                              NA  
861596                              NA  

[861597 rows x 13 columns]

# Pandas to spark df

In [5]:
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
df = spark.createDataFrame(pandas_df)
df.show()

+------+------------+--------+--------------------+----------+--------------------+-------------+----------+-----------------+------------+---------------+----------------+--------------------+
|    ID|SerialNumber|ListYear|        DateRecorded|      Town|             Address|AssessedValue|SaleAmount|       SalesRatio|PropertyType|ResidentialType|      NonUseCode|             Remarks|
+------+------------+--------+--------------------+----------+--------------------+-------------+----------+-----------------+------------+---------------+----------------+--------------------+
|815906|      170177|    2017|04/05/1999 12:00:...|New London|      293 PEQUOT AVE|       132440|    252500|            0.525|        null|     Two Family|            null|                null|
|     2|      900035|    2009|07/20/2010 12:00:...|   Andover|     1 DOGWOOD DRIVE|        55600|     99000|0.561616161616162| Vacant Land|             NA|              NA|                  NA|
|     3|       14011|    2014|

In [6]:
# spark.catalog.clearCache()
spark.stop()